In [ ]:
from callbird.src.datasets.naive_as_multi.NaiveAsMultiDataModule import NaiveAsMultiDataModule
from callbird.src.datasets.load_test_dataset import load_test_dataset
from callbird.src.ensure_torch_safe_globals import ensure_torch_safe_globals
from callbird.src.readUtils import readCommentedList, readLabeledMapping
from datasets import DatasetDict

dataset = load_test_dataset()

blacklist_naive = readCommentedList("/workspace/projects/callbird/datastats/test/blacklist_naive.txt")
dataset = dataset.filter(lambda x: x["ebird_code_and_call"] not in blacklist_naive)

def _process_loaded_multitask_data(dataset, decode: bool = True):
    def add_event_columns(example):
        example["detected_events"] = (example["start_time"], example["end_time"])
        # TODO: Fix event_cluster value
        example["event_cluster"] = [0]
        return example

    dataset = dataset.map(add_event_columns)

    def add_multilabel_column(example):
        example["ebird_code_multilabel"] = example["ebird_code_and_call"]
        return example
    
    dataset = dataset.map(add_multilabel_column)

    ebird_labels = set()
    for split in dataset.keys():
        ebird_labels.update(dataset[split]["ebird_code_and_call"])

    ebird_labels = sorted(list(ebird_labels))

    ebird_label_to_id = {lbl: i for i, lbl in enumerate(ebird_labels)}

    def label_to_id_fn(batch):
        for i in range(len(batch["ebird_code_multilabel"])):
            batch["ebird_code_multilabel"][i] = ebird_label_to_id[batch["ebird_code_multilabel"][i]]

        return batch

    dataset = dataset.map(
        label_to_id_fn,
        batched=True,
        batch_size=500,
        load_from_cache_file=True,
        num_proc=1,
    )

    return dataset

dataset = _process_loaded_multitask_data(dataset)

print(dataset["train"])


Resolving data files:   0%|          | 0/153 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['ebird_code', 'common_name', 'vocalization_type', 'start_time', 'end_time', 'filepath', 'short_call_type', 'ebird_code_and_call', 'detected_events', 'event_cluster', 'ebird_code_multilabel'],
        num_rows: 14933
    })
})
